In [ ]:
import warnings, os
import datetime as dt
import itertools as it
from numpy import nan, inf
import numpy as np
import pandas as pd

warnings.filterwarnings('ignore')
pd.set_option('display.width',       None)
pd.set_option('display.max_columns', None)

# Load

In [ ]:
Raw = pd.read_csv('Storage/Metatrader_WINN.csv', parse_dates=['datetime'])
Raw.head()

In [ ]:
def QUERY_SELECT(Raw, SYMBOLS, TFRAMES):
    return Raw[Raw['symbol'].isin(SYMBOLS) & Raw['tf'].isin(TFRAMES)] .reset_index(drop=1)

Query = QUERY_SELECT(Raw, SYMBOLS=['WIN$N'], TFRAMES=['D1'])
Query.head()

# Data

In [ ]:
def FORMAT_SOURCE(Query):
    Src = pd.DataFrame()
    Src[['A','Z']]                      = Query[['a','z']]
    Src[['Symbol','TF']]                = Query[['symbol','tf']]
    Src['Datetime']                     = Query['datetime']
    Src[['Ticks','Volume']]             = Query[['tick_volume','real_volume']]
    Src[['Open','High','Low','Close']]  = Query[['open','high','low','close']]
    Src[['Price']]                      = Query[['close']]

    Src['Change']   = (Src['Close'] - Src['Open'])
    Src['Hilo']     = (Src['High']  - Src['Low'])
    return Src 

Src = FORMAT_SOURCE(Query)
Src.head()

In [ ]:
def CALCULATIONS(Src):
    Calc = pd.DataFrame(Src)
    Calc[f'High +{1}']      = Calc['High']  .shift(-1)
    Calc[f'Low +{1}']       = Calc['Low']   .shift(-1)
    Calc[f'Close +{1}']     = Calc['Close'] .shift(-1)


    Calc['High in max']     =               Calc['High +1'] < Calc['High']
    Calc['Low in min']      = Calc['Low'] < Calc['Low +1']

    Calc['High out max']    =                  Calc['High'] < Calc['High +1']
    Calc['Low out min']     = Calc['Low +1'] < Calc['Low']


    Calc['Close in max']    =               Calc['Close +1'] < Calc['High']
    Calc['Close in min']    = Calc['Low'] < Calc['Close +1']

    Calc['Close out max']   =                    Calc['High'] < Calc['Close +1']
    Calc['Close out min']   = Calc['Close +1'] < Calc['Low']


    Calc['Break 0'] = (Calc['Close in min']  & Calc['Close in max'])   &  (Calc['Low in min']  & Calc['High in max'])
    Calc['Break 1'] = (Calc['Close in min']  & Calc['Close in max'])   &  (Calc['Low out min'] | Calc['High out max'])
    Calc['Break 2'] = (Calc['Close out min'] | Calc['Close out max'])  &  (Calc['Low out min'] | Calc['High out max'])
    return Calc


Calc = CALCULATIONS(Src)
Calc.head()

In [10]:
def STATS(Calc):
    BINS = [500,1000,1500,2000,2500,3000,3500,4000,4500,5000,5500,6000,6500,7000]
    pipe = []
    for b in BINS:
        Df = Calc[Calc['Hilo'] > b] #.reset_index(drop=1)
        pipe.append({
            'Hilo D0 >': b,
            'Count':    len(Df),
            'Inside':   np.round(Df['Break 0'].mean()*100, 1), 
            'Test':     np.round(Df['Break 1'].mean()*100, 1), 
            'Break':    np.round(Df['Break 2'].mean()*100, 1), 
        })
    pass
    return pd.DataFrame(pipe)

STATS(Calc)

,Hilo D0 >,Count,Inside,Test,Break
0,500,1245,11.6,35.1,52.5
1,1000,1181,12.2,35.7,51.2
2,1500,864,15.2,36.5,47.6
3,2000,493,18.9,33.3,47.1
4,2500,258,21.7,32.9,45.0
5,3000,140,25.7,31.4,42.1
6,3500,59,23.7,33.9,40.7
7,4000,31,16.1,32.3,48.4
8,4500,15,26.7,26.7,46.7
9,5000,6,33.3,33.3,33.3


# Snippets

In [ ]:
def CALCULATIONS(Src):
    Calc = pd.DataFrame(Src)
    Calc[f'High +{1}']      = Calc['High']  .shift(-1)
    Calc[f'Low +{1}']       = Calc['Low']   .shift(-1)
    Calc[f'Close +{1}']     = Calc['Close'] .shift(-1)


    Calc['High in max']     =               Calc['High +1'] < Calc['High']
    Calc['Low in min']      = Calc['Low'] < Calc['Low +1']

    Calc['High out max']    =                  Calc['High'] < Calc['High +1']
    Calc['Low out min']     = Calc['Low +1'] < Calc['Low']


    Calc['Close in max']    =               Calc['Close +1'] < Calc['High']
    Calc['Close in min']    = Calc['Low'] < Calc['Close +1']

    Calc['Close out max']   =                    Calc['High'] < Calc['Close +1']
    Calc['Close out min']   = Calc['Close +1'] < Calc['Low']


    Calc['Break 0'] = (Calc['Close in min']  & Calc['Close in max'])   &  (Calc['Low in min']  & Calc['High in max'])
    Calc['Break 1'] = (Calc['Close in min']  & Calc['Close in max'])   &  (Calc['Low out min'] | Calc['High out max'])
    Calc['Break 2'] = (Calc['Close out min'] | Calc['Close out max'])  &  (Calc['Low out min'] | Calc['High out max'])
    return Calc

In [ ]:
def STATS(Calc):
    BINS = [500,1000,1500,2000,2500,3000,3500,4000,4500,5000,5500,6000,6500,7000,7500,8000]
    pipe = []
    for b in BINS:
        Df = Calc[Calc['Hilo'] > b] #.reset_index(drop=1)
        pipe.append({
            'Hilo D0 >': b,
            'Count':    len(Df),
            'Inside':   np.round(Df['Break 0'].mean()*100, 1), 
            'Test':     np.round(Df['Break 1'].mean()*100, 1), 
            'Break':    np.round(Df['Break 2'].mean()*100, 1), 
        })
    pass
    return pd.DataFrame(pipe)

STATS(Calc)